In [1]:
%load_ext autoreload
%autoreload 2

from cnn import CNN
from dyn_cnn import DynCNN
from util import Util
from patch import Patch
from fixer import Fixer
from uglify import Uglify

import cPickle as pickle
import os
import mahotas as mh
import numpy as np
import time

import mlproof as mlp


from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 4007)
/home/d/nolearn/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
tifffile.py:1995: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
tifffile.py:1995: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
tifffile.py:1995: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)


In [2]:
DOJO_USER_DATA_FOLDER = '/home/d/Dropbox/DOJOSTUDY/'
DOJO_USERS_IDS = [11,13,18,37,40,43,5,50,53,62]
DOJO_USERS = []
for d in DOJO_USERS_IDS:
    DOJO_USERS.append('S'+str(d)+'tif')
    
def read_user(user, slice):
    img = mh.imread(DOJO_USER_DATA_FOLDER+user+os.sep+'z='+str(slice).zfill(8)+'.tif')
    img = Util.crop_by_bbox(img, dojo_bbox)
    return Util.frame_image(img)

def dojo_user_vi(user, slice, gt):
    img = read_user(user, slice)
    return Util.vi(img, gt)

def fix_dojo_slice(cnn, slice, input_image, input_prob, input_rhoana, input_gold,t=0.):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 sureness_threshold=t,
                                                 oversampling=False)

    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, best_sureness, vi_s, surenesses

def fix_dojo_slice_user_simulated(cnn, slice, input_image, input_prob, input_rhoana, input_gold, error_rate=0,t=0):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses, good_fixes, bad_fixes = Fixer.splits_user_simulated(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 error_rate=error_rate,
                                                 sureness_threshold=t,                                                 
                                                 oversampling=False)
    
    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, good_fixes, bad_fixes, merge_pairs, vi_s

def threed_vi(gt, seg):
    total_vi = 0
    slice_vi = []    
    for i in range(10):
        current_vi = Util.vi(gt[i].astype(np.int64), seg[i].astype(np.int64))
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi

def user_threed_vi(gt, user):
    total_vi = 0
    slice_vi = []
    for i in range(10):
        current_vi = dojo_user_vi(user, i, gt[i])
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi
    
def cnn_threed_vi(cnn, input_image, input_prob, input_rhoana, input_gold):
    total_vi = 0
    slice_vi = []
    slice_sureness = []
    for i in range(10):
        current_vi, sureness = fix_dojo_slice(cnn, i, input_image, input_prob, input_rhoana, input_gold)
        total_vi += current_vi
        slice_vi.append(current_vi)
        slice_sureness.append(sureness)
    total_vi /= 10
    return total_vi, slice_vi, slice_sureness
        

In [3]:
input_image, input_prob, input_gold, input_rhoana, dojo_bbox = Util.read_dojo_data() 

In [4]:
with open('/home/d/nets/MergeNet_larger_border_overlap_cylinder1_dataaugmentation/net.p', 'rb') as f:
    cnn = pickle.load(f)

In [4]:
# the new CNN
with open('/home/d/nets/MergeNet_cylinder1_larger_border_overlap_usevars/net.p', 'rb') as f:
    cnn = pickle.load(f)

In [5]:
with open('/home/d/dojo_xp/data/dojo_data_vis2014_patches_predicted.p', 'rb') as f:
    patches = pickle.load(f)
new_patches = []
for p in patches:
    p_new = {}
    p_new['image'] = p['image'] / 255.
    p_new['prob'] = p['prob'] / 255.
    p_new['merged_array'] = p['merged_array'] 
    p_new['border_overlap'] = p['border_overlap'] 
    new_patches.append(p_new)

In [13]:
# fix one slice a time with sureness threshold .95
vol_vi_s = []
for z in range(10):
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, input_image[z], input_prob[z], rhoana_after_merge_correction[z], input_gold[z], 
                                                 sureness_threshold=.95,
                                                 verbose=False)
    vol_vi_s.append(vi_s[-1])
    

In [14]:
np.median(vol_vi_s)

0.56186645959149706

In [15]:
vol_vi_s

[0.3737786615798342,
 0.394548115049429,
 0.4611100240446344,
 0.42946009966927434,
 0.45653426673149156,
 0.7313827147492598,
 0.6626228951383597,
 0.8701369111170498,
 0.7400183520925676,
 0.8747261709437684]

In [95]:
np.median(vol_vi_s)

0.49758863911900653

In [97]:
vol_vi_s

[0.38255406726973096,
 0.39475205397438984,
 0.37838898717369673,
 0.4271712450368943,
 0.3975991144058071,
 0.8186454056420018,
 0.7259866556400247,
 0.6007895383730038,
 0.5680060332011188,
 1.0475679727790466]

In [96]:
# fix one slice a time with sureness threshold .99
vol_vi_s2 = []
for z in range(10):
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, input_image[z], input_prob[z], rhoana_after_merge_correction[z], input_gold[z], 
                                                 sureness_threshold=.99,
                                                 verbose=False)
    vol_vi_s2.append(vi_s[-1])
np.median(vol_vi_s2)

0.43564467471374124

In [100]:
vol_vi_s2

[0.38255406726973096,
 0.39475205397438984,
 0.37838898717369673,
 0.4090421399893156,
 0.4144215663447275,
 0.456867783082755,
 0.6573175935752964,
 0.5398166489091327,
 0.5541351346259145,
 0.9300709422453526]

In [67]:
# simulate user with error rate 0 with limit of N corrections
bigM, out_vol, global_volume = Fixer.splits_global(cnn, input_image, input_prob, rhoana_after_merge_correction, input_gold)

476 generated in 7.67157196999 seconds.
Grouped into 119 patches in 0.00206613540649 seconds.
448 generated in 9.71588897705 seconds.
Grouped into 112 patches in 0.00202512741089 seconds.
472 generated in 9.95114207268 seconds.
Grouped into 117 patches in 0.00208902359009 seconds.
484 generated in 10.1509411335 seconds.
Grouped into 121 patches in 0.00217294692993 seconds.
520 generated in 10.9555058479 seconds.
Grouped into 130 patches in 0.00229096412659 seconds.
500 generated in 10.5159080029 seconds.
Grouped into 125 patches in 0.00238180160522 seconds.
480 generated in 10.139580965 seconds.
Grouped into 118 patches in 0.00216102600098 seconds.
476 generated in 9.9745631218 seconds.
Grouped into 117 patches in 0.00213503837585 seconds.
488 generated in 10.1972169876 seconds.
Grouped into 122 patches in 0.00215101242065 seconds.
552 generated in 11.2748889923 seconds.
Grouped into 138 patches in 0.00254106521606 seconds.


In [17]:
# with 15 secs per correction
user_simulated_vi = threed_vi(input_gold, out_vol)
np.median(user_simulated_vi[1])

0.4144289172505391

In [53]:
# with 5 secs per correction
user_simulated_vi = threed_vi(input_gold, out_vol)
np.median(user_simulated_vi[1])

0.40333873824924282

In [68]:
# with 5 secs per correction after merge error correction
user_simulated_vi = threed_vi(input_gold, out_vol)
np.median(user_simulated_vi[1])

0.3938181020766125

In [98]:
user_simulated_vi

(0.41273321981092315,
 [0.36469144760523253,
  0.3800517083459587,
  0.36488580658555847,
  0.3706597645645271,
  0.3973091017963517,
  0.3903271023568733,
  0.49296487907424513,
  0.47382503975664036,
  0.45760760586651106,
  0.4350097421573329])

In [51]:
out_vol

In [92]:
# random recommendations
bigM, out_vol, global_volume = Fixer.splits_global(cnn, input_image, input_prob, rhoana_after_merge_correction_random, input_gold, randomize=True)

488 generated in 7.87826013565 seconds.
Grouped into 122 patches in 0.00233793258667 seconds.
484 generated in 10.307518959 seconds.
Grouped into 120 patches in 0.00215411186218 seconds.
456 generated in 10.0021011829 seconds.
Grouped into 113 patches in 0.00202298164368 seconds.
484 generated in 10.1043610573 seconds.
Grouped into 121 patches in 0.00216197967529 seconds.
508 generated in 10.5845479965 seconds.
Grouped into 127 patches in 0.00231695175171 seconds.
512 generated in 10.6639130116 seconds.
Grouped into 128 patches in 0.00233888626099 seconds.
480 generated in 10.2517399788 seconds.
Grouped into 118 patches in 0.00215315818787 seconds.
460 generated in 9.76374006271 seconds.
Grouped into 113 patches in 0.00204205513 seconds.
500 generated in 10.3664739132 seconds.
Grouped into 125 patches in 0.00223994255066 seconds.
540 generated in 11.2085828781 seconds.
Grouped into 135 patches in 0.00239396095276 seconds.


In [93]:
random_vi = threed_vi(input_gold, out_vol)
np.median(random_vi[1]) 

0.47742953849063463

In [99]:
random_vi

(0.49098250120267145,
 [0.4595423295850365,
  0.4308474043605237,
  0.3986397439416596,
  0.39881363108964063,
  0.4774106038339303,
  0.47744847314733896,
  0.5331778295505849,
  0.5657477626512799,
  0.6298385736857668,
  0.5383586601809531])

In [30]:
# create bigM for the UI tool
bigM, a, global_volume = Fixer.splits_global(cnn, input_image, input_prob, input_rhoana, input_gold)

464 generated in 7.44634389877 seconds.
Grouped into 116 patches in 0.00208497047424 seconds.
448 generated in 9.60027599335 seconds.
Grouped into 112 patches in 0.00206613540649 seconds.
456 generated in 9.67844295502 seconds.
Grouped into 113 patches in 0.00205779075623 seconds.
484 generated in 10.0104978085 seconds.
Grouped into 121 patches in 0.00217604637146 seconds.
508 generated in 10.5294861794 seconds.
Grouped into 127 patches in 0.0022828578949 seconds.
488 generated in 10.2322368622 seconds.
Grouped into 122 patches in 0.0022189617157 seconds.
480 generated in 10.026018858 seconds.
Grouped into 118 patches in 0.00217914581299 seconds.
460 generated in 9.6668651104 seconds.
Grouped into 113 patches in 0.00210499763489 seconds.
488 generated in 10.0534870625 seconds.
Grouped into 120 patches in 0.00218987464905 seconds.
540 generated in 11.0391199589 seconds.
Grouped into 135 patches in 0.00241494178772 seconds.


In [31]:
with open('/home/d/dojo_xp/data/bigM_new_cnn.p', 'wb') as f:
    pickle.dump(bigM, f)

In [8]:
from uitools import UITools

In [9]:
merge_errors = UITools.get_top5_merge_errors(cnn, input_image, input_prob, input_rhoana)

working on slice 0
working on slice 1
working on slice 2
working on slice 3
working on slice 4
working on slice 5
working on slice 6
working on slice 7
working on slice 8
working on slice 9
merge error correction done after 1112.44038391 seconds


In [11]:
with open('/home/d/dojo_xp/data/merges_new_cnn.p', 'wb') as f:
    pickle.dump(merge_errors, f)

In [10]:
len(merge_errors)

19

In [47]:
with open(os.path.join('/home/d/dojo_xp/data/ui_out/new_cnn_test_with_saving_times/times.p'), 'rb') as f:
    c_t = pickle.load(f)
    c_t = [int(v) for v in c_t]
np.mean(c_t)

1889.0

In [12]:
#
# APPLY MERGE ERROR CORRECTION TO RHOANA
#
rhoana_after_merge_correction = np.array(input_rhoana)
# correct merge_errors if they are rated <.5
for me in merge_errors:
    pred = me[2]
    if pred < .5:
        z = me[0]
        label = me[1]
        border = me[3][0][1]
        a,b,c,d,e,f,g,h,i = UITools.get_merge_error_image(input_image[z], rhoana_after_merge_correction[z], label, border)        
        rhoana_after_merge_correction[z] = f
        print 'corrected', z

corrected 0
corrected 0
corrected 1
corrected 1
corrected 1
corrected 2
corrected 2
corrected 2
corrected 3
corrected 3
corrected 4
corrected 4
corrected 5
corrected 6
corrected 7
corrected 7
corrected 8
corrected 9
corrected 9


In [91]:
#
# APPLY MERGE ERROR CORRECTION TO RHOANA RANDOMLY
#
import random
rhoana_after_merge_correction_random = np.array(input_rhoana)
# correct merge_errors if they are rated <.5
for me in merge_errors:
    pred = me[2]
    if random.random() < .5:
        z = me[0]
        label = me[1]
        border = me[3][0][1]
        a,b,c,d,e,f,g,h,i = UITools.get_merge_error_image(input_image[z], rhoana_after_merge_correction_random[z], label, border)        
        rhoana_after_merge_correction_random[z] = f
        print 'corrected', z

corrected 0
corrected 0
corrected 1
corrected 1
corrected 1
corrected 1
corrected 5
corrected 5
corrected 8
corrected 8
